Fitche data 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.feature_selection import SelectKBest,f_classif
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.svm import SVR
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
path = "/Users/lait-zet/Desktop/Work_local/Data/data_livre.csv"

df = pd.read_csv(path)
olddata = df
df.head(4)
df.info()
df.isnull().sum()
df.duplicated().sum()
df.describe()

 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Order_ID                1000 non-null   int64  
 1   Distance_km             1000 non-null   float64
 2   Weather                 970 non-null    object 
 3   Traffic_Level           970 non-null    object 
 4   Time_of_Day             970 non-null    object 
 5   Vehicle_Type            1000 non-null   object 
 6   Preparation_Time_min    1000 non-null   int64  
 7   Courier_Experience_yrs  970 non-null    float64
 8   Delivery_Time_min       1000 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 70.4+ KB


,Order_ID,Distance_km,Preparation_Time_min,Courier_Experience_yrs,Delivery_Time_min
count,1000.000000,1000.000000,1000.000000,970.000000,1000.000000
mean,500.500000,10.059970,16.982000,4.579381,56.732000
std,288.819436,5.696656,7.204553,2.914394,22.070915
min,1.000000,0.590000,5.000000,0.000000,8.000000
25%,250.750000,5.105000,11.000000,2.000000,41.000000
50%,500.500000,10.190000,17.000000,5.000000,55.500000
75%,750.250000,15.017500,23.000000,7.000000,71.000000
max,1000.000000,19.990000,29.000000,9.000000,153.000000


In [2]:
df.isnull().sum()

Order_ID                   0
Distance_km                0
Weather                   30
Traffic_Level             30
Time_of_Day               30
Vehicle_Type               0
Preparation_Time_min       0
Courier_Experience_yrs    30
Delivery_Time_min          0
dtype: int64

In [3]:
def handle_missing(df, columns):
    i = 0
    while i < len(columns):
        col = columns[i]
        if df[col].dtype == 'object':
            df[col] = df[col].fillna(df[col].mode()[0])
        else:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df[col] = df[col].fillna(df[col].median())
        i += 1
    return df
convert_columns = ['Weather', 'Traffic_Level', 'Time_of_Day', 'Courier_Experience_yrs']
df = handle_missing(df, convert_columns)
# df.isnull().sum()

In [4]:
def is_numeric(df):
    return df.select_dtypes(include=['int64', 'float64'])

def is_categorie(df):
    return df.select_dtypes(exclude=['int64', 'float64'])
Data_Numeric = is_numeric(df)
Data_categorie = is_categorie(df)


Visulisation

In [5]:
# corr = Data_Numeric.corr()
# plt.figure(figsize=(10, 6))
# sns.heatmap(corr, vmax=1 , vmin=-1 , annot=True)
# plt.title('Heatmap de corrélation des variables numériques')
# plt.show()

In [6]:
# categorical_cols = ['Weather', 'Traffic_Level','Time_of_Day', 'Vehicle_Type']
# olddata
# for col in categorical_cols:
#     sns.countplot(x = col , data = olddata , palette='husl')
#     plt.title(f"distribution : {col}")
#     plt.xlabel(col)
#     plt.ylabel("Count")
#     plt.show()



In [7]:
# for cols in categorical_cols:
#     sns.boxplot(x = cols ,y = 'Delivery_Time_min' ,data = df ,palette='Set2')
#     plt.title(f"Relation between {cols} and Dilvery time")
#     plt.xlabel(col)
#     plt.ylabel("Delivery Time")
#     plt.show()

In [8]:
# plt.figure(figsize=(6,4))
# sns.histplot(df['Delivery_Time_min'], bins=20, kde=True, color='skyblue')
# plt.title("Distribution de la variable cible : Delivery_Time_min")
# plt.xlabel("Delivery Time (min)")
# plt.ylabel("Fréquence")
# plt.show()

Split_data


In [9]:
num_data = Data_Numeric[['Distance_km','Preparation_Time_min']]
cat_data =Data_categorie[['Weather','Traffic_Level']]
def use_scled(data):
    le = StandardScaler()
    scaled = le.fit_transform(data)
    scaled_numerique = pd.DataFrame(scaled, columns=data.columns)
    return scaled_numerique
def use_hote(data):
    la = OneHotEncoder(sparse_output=False)
    encoded = la.fit_transform(data)
    cooder_categorie = pd.DataFrame(encoded,columns = la.get_feature_names_out(data.columns))
    return cooder_categorie
cooder_categorie = use_hote(cat_data)
scaled_numerique = use_scled(num_data)
# cooder_categorie


In [10]:
target = df['Delivery_Time_min']
prepared_data = pd.concat(
    [scaled_numerique, cooder_categorie, target],
    axis=1
)
X = df.drop(columns=["Delivery_Time_min"])
Y = df['Delivery_Time_min']
def split_data(df):
    X = df.drop(columns=["Delivery_Time_min"])
    Y = df['Delivery_Time_min']
    train_X , test_X , train_Y , test_Y = train_test_split(
    X , Y , test_size=0.2, random_state=42
    )
    return train_X,test_X,train_Y,test_Y

train_X,test_X,train_Y,test_Y = split_data(prepared_data)

SelectKBest

In [ ]:
# selector = SelectKBest(score_func = f_classif ,k=3)
# x_new = selector.fit_transform(X,Y)
# mask = selector.get_support()
# mask
# for col, score in zip(X.columns, selector.scores_):
#     print(f"{col}: {score:.2f}")


ValueError: could not convert string to float: 'Windy'

GridSearchCV

In [12]:
def GridSearch_CV(trainX, testX ,trainY ,testY):
    RF = RandomForestRegressor()
    param_grid = {
        'n_estimators': [100, 200, 500],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2', 1.0]
    }
    grid_search_for_RF = GridSearchCV(
        estimator = RF,
        param_grid = param_grid,
        scoring='r2',
        cv = 5,
        n_jobs=-1,
        verbose=1
    )
    grid_search_for_RF.fit(train_X,train_Y)
    best_RF = grid_search_for_RF.best_estimator_



    svr = SVR()
    param_grid = {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf'],
        'gamma': ['scale', 'auto']
    }
    grid_search_for_SVC = GridSearchCV(
        estimator = svr,
        param_grid = param_grid,
        cv = 5,
        scoring='r2',
        verbose= 0
    )
    grid_search_for_SVC.fit(train_X,train_Y)
    best_SVR = grid_search_for_SVC.best_estimator_



    pred_RF = best_RF.predict(test_X)
    pred_SVR = best_SVR.predict(test_X)

    mae_RF = mean_absolute_error(test_Y, pred_RF)
    r2_RF = r2_score(test_Y, pred_RF)

    mae_SVR = mean_absolute_error(test_Y, pred_SVR)
    r2_SVR = r2_score(test_Y, pred_SVR)

    print(f"Random Forest - MAE: {mae_RF:.3f}, R²: {r2_RF:.3f}")
    print(f"SVR            - MAE: {mae_SVR : .3f}, R²: {r2_SVR:.3f}")

    if r2_RF > r2_SVR:
        print("\n final choisi est: Random Forest Regressor")
    else:
        print("\nfinal choisi est: SVR (Support Vector Regressor)")
    return {
        "RandomForestRegressor" : {"MAE":mae_RF, "r_2" : r2_RF},
        "SVR" : {"MAE":mae_RF, "r_2" : r2_SVR}
    }
result = GridSearch_CV(train_X,test_X, train_Y,test_Y)

Fitting 5 folds for each of 243 candidates, totalling 1215 fits
Random Forest - MAE: 6.982, R²: 0.790
SVR            - MAE:  5.990, R²: 0.817

final choisi est: SVR (Support Vector Regressor)


Pipline